In [30]:
#https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html?highlight=autoreload
%load_ext autoreload
%autoreload 2

import json
import time
import uuid
from datetime import datetime
import pandas as pd

from Kafka import KafkaConfluentWriter, KafkaConfluentReader
from OpenWeatherMap import OpenWeatherMap

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
kafkaConfluentWriter = KafkaConfluentWriter('weather.cleaned')
kafkaConluentReaderForecast = KafkaConfluentReader('weather.forecast', True)
kafkaConfluentReaderCleaned = KafkaConfluentReader('weather.cleaned', False)

def clean_forecast_data() -> None:    
    try:
        print("Looking for new unique data ...")

        while True:
            msg = kafkaConluentReaderForecast.poll(1.0)
            
            if msg is not None:
                lastCleaned = kafkaConfluentReaderCleaned.get_latest_message(1.0)
                if lastCleaned is None:
                    kafkaConfluentWriter.produce(msg.key(), msg.value())
                    print("Stored first data!")
                elif msg.value() != lastCleaned.value():
                    kafkaConfluentWriter.produce(msg.key(), msg.value())
                    print("Stored new data!")
                else:
                    print("Nothing to do!")

    except KeyboardInterrupt:
        print("... search for new values stopped!")

In [32]:
clean_forecast_data()

Looking for new unique data ...
Message delivered to weather.cleaned [0]
Stored new data!
... search for new values stopped!
